In [2]:
import re
import datetime
import pandas as pd

def read_sql_results(file, regex):
    lines = open(file, 'r').readlines()
    pattern = re.compile(regex)
    lines = list(filter(lambda line: pattern.match(line), lines))

    tables = list(map(lambda line: strip(line.replace('\n', '').split('|')), lines))
    
    columns = tables.pop(0)

    return pd.DataFrame(tables, columns=columns)

def strip(lines):
    return list(map(lambda line: line.strip(), lines))

In [3]:
lessons = read_sql_results('data/tech_quality/lessons.txt', '.+\|.+\|.+\|.+')
participants = read_sql_results('data/tech_quality/participants.txt', '.+\|.+')
quality = read_sql_results('data/tech_quality/quality.txt', '.+\|.+')
users = read_sql_results('data/tech_quality/users.txt', '.+\|.+')

In [4]:
users = users.drop_duplicates()
participants = participants.drop_duplicates()
lessons = lessons.drop_duplicates()

users = users.rename(columns={'id': 'user_id'})
lessons = lessons.rename(columns={'id': 'lesson_id'})
lessons['date'] = pd.to_datetime(lessons['scheduled_time'], utc=True).dt.tz_convert('Europe/Moscow').dt.date

In [5]:
quality = quality[quality['tech_quality'] != '']
quality['tech_quality'] = quality['tech_quality'].astype(float)

In [6]:
lessons = lessons[lessons.subject == 'phys']
lessons = lessons.merge(participants, on='event_id')
lessons = lessons.merge(users, on='user_id')

# before joining scores we must drop all unnecessary rows with repeating lesson_ids
lessons = lessons[lessons['role'] == 'tutor']

lessons = lessons.merge(quality, on='lesson_id')

In [7]:
date_user_avgs = lessons.groupby(['date', 'user_id'])['tech_quality'].mean().reset_index()

In [8]:
date_user_avgs

,date,user_id,tech_quality
0,2020-01-11,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,5.000000
1,2020-01-11,30a19496-bdaf-461c-abbc-2709ae520201,5.000000
2,2020-01-11,43efce48-94b2-4412-857f-223d45969008,5.000000
3,2020-01-11,603b8641-c6f6-4d89-ac89-88e50d45aa0d,4.800000
4,2020-01-11,8fe03f08-8581-430c-a590-9888ab36deb3,4.428571
...,...,...,...
59,2020-01-20,30a19496-bdaf-461c-abbc-2709ae520201,4.500000
60,2020-01-20,43efce48-94b2-4412-857f-223d45969008,4.500000
61,2020-01-20,696c838e-c054-4e9f-a51a-50bf5660f364,5.000000
62,2020-01-20,b37ccae8-fc31-4ad8-8f55-ca855b23fbf6,5.000000


In [9]:
date_mins = date_user_avgs.loc[date_user_avgs.groupby(['date'])['tech_quality'].idxmin()]
date_mins['tech_quality'] = date_mins['tech_quality'].round(2)
date_mins

,date,user_id,tech_quality
4,2020-01-11,8fe03f08-8581-430c-a590-9888ab36deb3,4.43
10,2020-01-12,696c838e-c054-4e9f-a51a-50bf5660f364,4.89
13,2020-01-13,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,5.00
25,2020-01-14,c6718d0e-976c-4d6c-b0e0-32c770776567,4.00
26,2020-01-15,603b8641-c6f6-4d89-ac89-88e50d45aa0d,5.00
30,2020-01-16,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,4.00
39,2020-01-17,696c838e-c054-4e9f-a51a-50bf5660f364,4.50
46,2020-01-18,43efce48-94b2-4412-857f-223d45969008,4.25
57,2020-01-19,be676776-8366-4c71-8a35-d58014806eb5,4.50
59,2020-01-20,30a19496-bdaf-461c-abbc-2709ae520201,4.50
